In [14]:
# -- driver_run_forecast_LV1.py  --
# master python script to do a full LV1 forecast simulation

import sys
import pickle
import numpy as np
import os
from datetime import datetime, timedelta
import gc
import resource
import subprocess

##############

sys.path.append('../sdpm_py_util')

import atm_functions as atmfuns
import ocn_functions as ocnfuns
import grid_functions as grdfuns
import plotting_functions as pltfuns
from get_PFM_info import get_PFM_info
import init_funs as infuns
from make_LV1_dotin_and_SLURM import make_LV1_dotin_and_SLURM
from run_slurm_LV1 import run_slurm_LV1

print('\nStarting the LV1 simulation, Current time ', datetime.now())

# we are going to make a forecast
run_type = 'forecast'

# PFM has all of the information needed to run the model
clean_start = True
infuns.initialize_simulation(clean_start)
PFM=get_PFM_info()

# return the hycom forecast date based on the PFM simulation times
#yyyymmdd_hy = infuns.determine_hycom_foretime() 

t1  = PFM['fetch_time']                              # this is the first time of the PFM forecast
# now a string of the time to start ROMS (and the 1st atm time too)
yyyymmddhhmm_pfm = "%d%02d%02d%02d%02d" % (t1.year, t1.month, t1.day, t1.hour, t1.minute)
t2  = t1 + PFM['forecast_days'] * timedelta(days=1)  # this is the last time of the PFM forecast

print('going to do a PFM forecast from')
print(t1)
print('to')
print(t2)
print('\n')

# get_hycom_foretime is a function that returns the lates hycom forecast date with data to span the PFM forecast from 
# this functions cleans, then refreshes the hycom directory.
yyyymmdd_hy = ocnfuns.get_hycom_foretime((t1,t2))


# now make the catted hycom data .nc file
#ocnfuns.cat_hycom_to_onenc(yyyymmdd_hy,[t1,t2])
print('we will use the hycom forecast from')
print(yyyymmdd_hy)
print('for the PFM forecast.\n')

print('catting the hycom.nc files into one...')
ocnfuns.cat_hycom_to_onenc(yyyymmdd_hy,[t1,t2])
print('...done.\n')

print('making the hycom pickle files from catted hycom.nc...')
ocnfuns.hycom_cat_to_pickles(yyyymmdd_hy)
print('...done.\n')


use_ncks = 1 # flag to get data using ncks. if =0, then a pre saved pickle file is loaded.
use_pckl_sav = 1
sv_ocnR_pkl_file=1
fr_ocnR_pkl_file=1
frm_ICpkl_file = 1
frm_BCpkl_file = 1
fr_ocnR_pkl_file=1

# what are we going to plot?
plot_ocn = 1
plot_ocnr = 1
plot_atm = 1
plot_all_atm = 1
plot_ocn_icnc= 1
load_plot_atm= 1

# need the file names and locations of the pickle and .nc files we will save
fn_pckl = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ocn_tmp_pckl_file']
ocnIC_pckl = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ocnIC_tmp_pckl_file']
ic_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ini_file']
bc_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_bc_file']
ocnBC_pckl = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ocnBC_tmp_pckl_file']
fn_atm_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_atm_file'] # LV1 atm forcing filename


t0 = datetime.now()

if plot_ocn ==1:
    print('making some plots from: ' + fn_pckl)
    cmd_list = ['python','-W','ignore','plotting_functions.py','plot_ocn_fields_from_dict_pckl',fn_pckl]
    os.chdir('../sdpm_py_util')
    ret1 = subprocess.run(cmd_list)     
    #pltfuns.plot_ocn_fields_from_dict_pckl(fn_pckl)
    print('subprocess return code? ' + str(ret1.returncode) +  ' (0=good)')
    print('...done')
    os.chdir('../driver')

t1= datetime.now()
print('this took:')
print(t1-t0)
print('\n')



# put the ocn data on the roms grid
print('starting: ocnfuns.hycom_to_roms_latlon(OCN,RMG)')
if sv_ocnR_pkl_file==0:
    print('returning OCN_R')
    OCN_R  = ocnfuns.hycom_to_roms_latlon(OCN,RMG)
else:
    os.chdir('../sdpm_py_util')
    print('putting the hycom data in ' + fn_pckl + ' on the roms grid...')
    cmd_list = ['python','-W','ignore','ocn_functions.py','make_all_tmp_pckl_ocnR_files',fn_pckl]
    os.chdir('../sdpm_py_util')
    ret1 = subprocess.run(cmd_list)     
    #ocnfuns.make_all_tmp_pckl_ocnR_files(fn_pckl)
    os.chdir('../driver')
    print('subprocess return code? ' + str(ret1.returncode) +  ' (0=good)')


cmd_list = ['python','-W','ignore','ocn_functions.py','print_maxmin_HYrm_pickles']
os.chdir('../sdpm_py_util')
ret1 = subprocess.run(cmd_list)     
os.chdir('../driver')
#ocnfuns.print_maxmin_HYrm_pickles()
print('driver_run_forecast_LV1: done with hycom_to_roms_latlon')
# add OCN + OCN_R plotting function here !!!

## plot OCN_R
if plot_ocnr == 1:
#    pltfuns.plot_ocn_R_fields_pckl()
    print('plotting LV1 ocn_R_fields...')
    cmd_list = ['python','-W','ignore','plotting_functions.py','plot_ocn_R_fields_pckl']
    os.chdir('../sdpm_py_util')
    ret1 = subprocess.run(cmd_list)     
    os.chdir('../driver')
    print('subprocess return code? ' + str(ret1.returncode) +  ' (0=good)')

t2 = datetime.now()
print('...done with LV1 ocn_R')
print('this took:')
print(t2-t1)
print('\n')



# make the depth pickle file
print('making the depth pickle file...')
fname_depths = PFM['lv1_forc_dir'] + '/' + PFM['lv1_depth_file']
cmd_list = ['python','-W','ignore','ocn_functions.py','make_rom_depths',fname_depths]
os.chdir('../sdpm_py_util')
ret6 = subprocess.run(cmd_list)     
os.chdir('../driver')
print('subprocess return code? ' + str(ret6.returncode) +  ' (0=good)')
print('\n')


if fr_ocnR_pkl_file==0:
    OCN_IC = ocnfuns.ocn_r_2_ICdict(OCN_R,RMG,PFM)
else:
    print('going to save OCN_IC to a pickle file: ' + ocnIC_pckl)
    os.chdir('../sdpm_py_util')
    cmd_list = ['python','-W','ignore','ocn_functions.py','ocn_r_2_ICdict_pckl',ocnIC_pckl]
    ret3 = subprocess.run(cmd_list)     
    os.chdir('../driver')
    print('OCN IC data saved with pickle, correctly? ' + str(ret3.returncode) + ' (0=yes,1=no)')

print('driver_run_forecast_LV1: done with ocn_r_2_ICdict')
# add OCN_IC.nc plotting function here !!!!
t4 = datetime.now()
print('this took:')
print(t4-t2)
print('\n')



if frm_ICpkl_file == 0:
    print('making IC file: '+ ic_file_out)
    ocnfuns.ocn_roms_IC_dict_to_netcdf(OCN_IC, ic_file_out)
else:
    print('making IC file from pickled IC: '+ ic_file_out)
    cmd_list = ['python','-W','ignore','ocn_functions.py','ocn_roms_IC_dict_to_netcdf_pckl',ocnIC_pckl,ic_file_out]
    os.chdir('../sdpm_py_util')
    ret4 = subprocess.run(cmd_list)     
    os.chdir('../driver')
    print('OCN IC nc data saved, correctly? ' + str(ret4.returncode) + ' (0=yes)')

print('done makeing IC file.')

if plot_ocn_icnc == 1:
    pltfuns.plot_ocn_ic_fields(ic_file_out)

t5 = datetime.now()
print('this took:')
print(t5-t4)
print('\n')



# get the OCN_BC dictionary
if fr_ocnR_pkl_file==0:
    OCN_BC = ocnfuns.ocn_r_2_BCdict(OCN_R,RMG,PFM)
else:
    print('going to save OCN_BC to a pickle file to:')
    ocnBC_pckl = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ocnBC_tmp_pckl_file']
    print(ocnBC_pckl) 
    os.chdir('../sdpm_py_util')
    cmd_list = ['python','-W','ignore','ocn_functions.py','ocn_r_2_BCdict_pckl_new',ocnBC_pckl]
    ret4 = subprocess.run(cmd_list)     
    os.chdir('../driver')
    print('OCN BC data saved with pickle, correctly? ' + str(ret4.returncode) + ' (0=yes)')
    
t6 = datetime.now()
print('this took:')
print(t6-t5)
print('\n')


if frm_BCpkl_file == 0:
    print('making BC file: '+ bc_file_out)
    ocnfuns.ocn_roms_BC_dict_to_netcdf(OCN_BC, bc_file_out)
else:
    print('making BC nc file from pickled BC: '+ bc_file_out)
    cmd_list = ['python','-W','ignore','ocn_functions.py','ocn_roms_BC_dict_to_netcdf_pckl',ocnBC_pckl,bc_file_out]
    os.chdir('../sdpm_py_util')
    ret5 = subprocess.run(cmd_list)     
    os.chdir('../driver')
    print('OCN BC nc data saved, correctly? ' + str(ret5.returncode) + ' (0=yes)')

print('done makeing BC nc file.')
t7 = datetime.now()
print('this took:')
print(t7-t6)
print('\n')


# now for the atm part...
cmd_list = ['python','-W','ignore','atm_functions.py','get_atm_data_as_dict']
os.chdir('../sdpm_py_util')
ret5 = subprocess.run(cmd_list)   
print('return code: ' + str(ret5.returncode) + ' (0=good)')  
os.chdir('../sdpm_py_util')
print('...done.')
t8 = datetime.now()
print('this took:')
print(t8-t7)
print('\n')

# plot some stuff
if plot_atm == 1:
    pltfuns.plot_atm_fields()
    print('done with plotting ATM fields')

t9 = datetime.now()
print('this took:')
print(t9-t8)
print('\n')

level = 1
# put the atm data on the roms grid, and rotate the velocities
# everything in this dict turn into the atm.nc file
print('in atmfuns.get_atm_data_on_roms_grid(ATM,RMG)')
#ATM_R  = atmfuns.get_atm_data_on_roms_grid(ATM,RMG)
cmd_list = ['python','-W','ignore','atm_functions.py','get_atm_data_on_roms_grid',str(level)]
os.chdir('../sdpm_py_util')
ret5 = subprocess.run(cmd_list)   
print('return code: ' + str(ret5.returncode) + ' (0=good)')  
os.chdir('../sdpm_py_util')
print('done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)')
# all the fields plotted with the data on roms grid

if plot_all_atm == 1:
#    pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)
    pltfuns.plot_all_fields_in_one(str(level))
    print('done with: pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)')

t10 = datetime.now()
print('this took:')
print(t10-t9)
print('\n')

# fn_out is the name of the atm.nc file used by roms
print('driver_run_forcast_LV1: saving ATM file to ' + fn_atm_out)
#atmfuns.atm_roms_dict_to_netcdf(ATM_R,fn_atm_out)
cmd_list = ['python','-W','ignore','atm_functions.py','atm_roms_dict_to_netcdf',str(level)]
os.chdir('../sdpm_py_util')
ret5 = subprocess.run(cmd_list)   
print('return code: ' + str(ret5.returncode) + ' (0=good)')  
os.chdir('../sdpm_py_util')
print('driver_run_forecast_LV1:  done with writing ATM.nc file.') 
# put in a function to plot the atm.nc file if we want to

if load_plot_atm == 1:
    pltfuns.load_and_plot_atm(str(level))
    print('done with pltfuns.load_and_plot_atm(PFM)')

t11 = datetime.now()
print('this took:')
print(t11-t10)
print('\n')


print('driver_run_forecast_LV1:  now make .in and .sb files')

pfm_driver_src_dir = os.getcwd()

yyyymmdd = PFM['yyyymmdd']
hhmm = PFM['hhmm']

os.chdir('../sdpm_py_util')
make_LV1_dotin_and_SLURM( PFM , yyyymmdd + hhmm )
print('...done')

# run command will be
print('now running roms with slurm')
run_slurm_LV1(PFM)

os.chdir('../driver')

t12 = datetime.now()
print('this took:')
print(t12-t11)
print('\n')

# now making history file plots
print('\nFinished the LV1 simulation')
print('now making LV1 history file plots')
pltfuns.make_all_his_figures('LV1')
print('Current time: ', datetime.now())

#######################





Starting the LV1 simulation, Current time  2024-09-23 15:06:53.509810
we are going to start clean...
getting PFM info...
removing PFM info file...
now removing all input files...
now making a new PFM.pkl file.
PFM info was saved as /scratch/PFM_Simulations/PFM_run_info.pkl


In [12]:
print(PFM['yyyymmdd'])
print(PFM['hhmm'])

20240923
1200


In [ ]:


fetch_time = PFM['fetch_time'] # the start of the PFM forecast
hy_time = datetime(fetch_time.year,fetch_time.month,fetch_time.day)
hy_time = hy_time - timedelta(days=1)
yyyymmdd = "%d%02d%02d" % (hy_time.year, hy_time.month, hy_time.day)

print(PFM['forecast_days'])

t1  = fetch_time                                     # this is the first time of the PFM forecast
t2  = t1 + PFM['forecast_days'] * timedelta(days=1)  # this is the last time of the PFM forecast

print('making a PFM simulation starting from')
print(t1)
print('and ending at')
print(t2)
print('getting the right hycom data...')
# make stuff below this into a function...
# n0 is the number of files we should have
# num_missing is the number of files we are missing
n0, num_missing = ocnfuns.check_hycom_data(yyyymmdd,[t1,t2])
print('there should be ' + str(n0) + ' files.')
print('there are ' + str(num_missing) + ' missing files.')



In [10]:


if num_missing == 0:
    print('we have the hycom files from the ' + yyyymmdd )
    print('hycom forecast to run the PFM forecast')
    ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])
    # cat hycom files to one .nc.
else:
    # try downloading all of the data again
    print('we did not have the files from the ' + yyyymmdd + ' hycom forecast')
    print('but we will try getting that data again. Maybe hycom filled in their .nc files?')
    ocnfuns.get_hycom_data(yyyymmdd)
    n02, num_missing2 = ocnfuns.check_hycom_data(yyyymmdd,[t1,t2])
    if num_missing2 == 0:
        print('Hycom now has the data we need to run the PFM forecast')
        print('catting the hycom.nc files into one...')
        ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])
        print('...done')
    else:
        print('we need the change the date of the hycom forecast (but not change the dates of the PFM forecast)')
        # yyyymmdd = yyyymmdd - 1 day
        # do this over again!



we did not have the files from the 20240917hycom forecast
but we will try getting that data again. Maybe hycom filled in their .nc files?


ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

...

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-17T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 19:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 282 <= time <= 282.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 285 <= time <= 285.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 294 <= time <= 294.5 does not fall within valid coordinate range 192 <= time <= 228
ERROR: nco_get_vara() failed to nc_get_vara() variable "water_u"
ERROR NC_EACCESS Access failure
HINT: NC_EACCESS errors signify a problem receiving data from a DAP server. This can occur in multiple ways, including 1) When NCO requests (with nco_var_get()) more data than the server is configured to dispense at one time. A workaround for this might be to request data in smaller amounts at one time. This can be accomplished by accessing hyperslabs or multi-slabs of data as described at http://nco.sf.net/nco.html#mlt. 2) When NCO requests data from a buggy DAP library. The DAP software built i

oc_open: server error retrieving url: code=3 message="Invalid Parameter Exception: DArrayDimension.setProjection: Bad Projection Request: stop >= size: 32:13"nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco_get_vara()
nco_err_exit(): ERROR Error code is -77. Translation into English with nc_strerror(-77) is "NetCDF: Access failure"


ncks: ERROR User-specified coordinate value range 300 <= time <= 300.5 does not fall within valid coordinate range 1.66006e-321 <= time <= 4.63444e-310
ncks: ERROR User-specified coordinate value range 324 <= time <= 324.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 306 <= time <= 306.5 does not fall within valid coordinate range 1.58101e-321 <= time <= 4.63444e-310
ncks: ERROR nco_lmt_evl_dmn_crd() reports domain 3.420000000e+02 <= time <= 3.425000000e+02 brackets no coordinate values
ncks: INFO user-specified coordinate minimum: "2024-09-23T18:00"
ncks: INFO user-specified coordinate maximum: "2024-09-23T18:30"
ncks: ERROR User-specified coordinate value range 333 <= time <= 333.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate value range 330 <= time <= 330.5 does not fall within valid coordinate range 192 <= time <= 228
ncks: ERROR User-specified coordinate valu

In [3]:
ocnfuns.cat_hycom_to_onenc(yyyymmdd,[t1,t2])


1

In [3]:

ocnfuns.get_hycom_data('20240922')


ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: INFO Will now attempt wget on the full filepath. wget will fail if the file is "hidden" behind a DAP server. Unfortunately, failed wget attempts creates rather long pathnames in the current directory. fxm TODO nco970, nco971. On the other hand, wget should succeed if the file is stored in any publicly-accessible web location.
ncks: INFO Unable to find file /tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z on local system. Found hsi command indicating presence of High Performance Storage System (HPSS). Will assume file is stored on HPSS. Received no local path information and so will try to derive suitable local filepath from given filename...
ncks: INFO Retrieved files will be stored in derived directory

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

In [5]:
yyyymmdd='20240922'
yyyy = yyyymmdd[0:4]
mm = yyyymmdd[4:6]
dd = yyyymmdd[6:8]
dstr0 = yyyy + '-' + mm + '-' + dd + 'T12:00' # this is the forecast time

t1 =  datetime.strptime(dstr0,"%Y-%m-%dT%H:%M")
t2 = t1 + 8.0 * timedelta(days=1)
times = [t1,t2]
n0, num_missing = ocnfuns.check_hycom_data(yyyymmdd,times)

print('there should be')
print(str(n0))
print('files.')
print('The number of missing files is:')
print(str(num_missing))
print('(0 means we have all files we checked for)')



/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-22T12:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-22T15:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-22T18:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-22T21:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T00:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T03:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T06:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T09:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T12:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T15:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T18:00.nc
/scratch/PFM_Simulations/hycom_data/hy_ssh_2024-09-22T12:00_2024-09-23T21:00.nc
/scratch/PFM_Simulations/hycom_data/hy_s

In [16]:
import glob
hycom_dir = '/scratch/PFM_Simulations/hycom_data/'
fns = glob.glob(hycom_dir + '*.nc')

#print(fns)
#print(fns[0])
fns3 = []
for fns2 in fns:
    #print(fns2[43:53])
    fns3.append(fns2[43:53])

#print(fns3)
set_res = set(fns3)
list_res = list(set_res)
list_res2 = sorted(list_res)
print(list_res2)

['2024-09-12', '2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16', '2024-09-17', '2024-09-21', '2024-09-22']


In [2]:
# this function removes all unneeded hycom forecasts. Only keeps the latest forecast with 8 days of data
# ie. clean_hycom_dir leave ONLY the hycom forecast that will be used in PFM
ocnfuns.clean_hycom_dir()
ocnfuns.refresh_hycom_dir()


there is only one hycom date in the directoy
we are not deleting any files.
we will download, hopefully, the hycom with these forecast dates:
['20240922', '20240923']
downloading hycom 20240922 forecast...


ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_sshruns/FMRC_ESPC-D-V02_ssh_RUN_2024-09-22T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

...

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"
ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not sear

ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"


ERROR: nco__open() unable to open file "tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z"
ERROR NC_ENOTNC Not a netCDF file
HINT: NC_ENOTNC errors can occur for many reasons. If your use-case matches one of the five listed below, take the corrective action indicated:
1. The netCDF file is truly corrupt, e.g., truncated. To verify whether the file is corrupt, try, e.g., printing its metadata with 'ncks -m in.nc'. This test will likely fail on a corrupt file
2. An NCO operator linked only to the classic netCDF3 library attempts to read netCDF4 (or HDF5) files. However, this executable seems to have been built with the capability to manipulate netCDF4 files, so it is unlikely that this command failed only because the input datasets are netCDF4 format. Something else is going wrong. 
3. NCO attempts to read other filetypes (HDF4, HDF-EOS2, PnetCDF/CDF5) for which support must be (but was not) enabled at netCDF build-time. NCO can access HDF4 

ncks: INFO DAP-access to https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z failed with error code -90. Translation into English with nc_strerror(-90) is "NetCDF: file not found"
ncks: WARNING not searching for https://tds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z on remote filesystem, using local file ds.hycom.org/thredds/dodsC/FMRC_ESPC-D-V02_v3zruns/FMRC_ESPC-D-V02_v3z_RUN_2024-09-23T12:00:00Z instead
nco_err_exit(): ERROR Short NCO-generated message (usually name of function that triggered error): nco__open()
nco_err_exit(): ERROR Error code is -51. Translation into English with nc_strerror(-51) is "NetCDF: Unknown file format"


In [4]:

ocnfuns.clean_hycom_dir()
hy_dates = ocnfuns.stored_hycom_dates()
fn_miss_list = ocnfuns.get_missing_file_num_list(hy_dates)

print(hy_dates)
print(fn_miss_list)


the date 2024-09-21 will not be deleted
the date 2024-09-22 will be deleted
['/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-22T12:00_2024-09-23T15:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-22T12:00_2024-09-24T18:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-22T12:00_2024-09-28T03:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-22T12:00_2024-09-25T00:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-22T12:00_2024-09-22T21:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-22T12:00_2024-09-26T06:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_t3z_2024-09-22T12:00_2024-09-24T03:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-22T12:00_2024-09-22T18:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_s3z_2024-09-22T12:00_2024-09-27T12:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_v3z_2024-09-22T12:00_2024-09-26T12:00.nc', '/scratch/PFM_Simulations/hycom_data/hy_u3z_2024-09-22T12:00_2024-09-25T06:00.nc', '/scratch/

In [ ]:

t0s = ocnfuns.stored_hycom_dates()
dstr0 = t0s[0] + 'T12:00' # this is the forecast time
t0 =  datetime.strptime(dstr0,"%Y-%m-%dT%H:%M")
tnow = datetime.now()
tget = []
while t0<tnow - timedelta(days=1):
    t0 = t0 + timedelta(days=1)
    yyyymmdd = "%d%02d%02d" % (t0.year, t0.month, t0.day)
    tget.append(yyyymmdd)

print('we will download, hopefully, the hycom with these forecast dates:')
print(tget)
for tt in tget:
    print('downloading hycom ' + tt + ' forecast')
    



In [ ]:

import glob
hycom_dir = '/scratch/PFM_Simulations/hycom_data/'

hy_dates = ocnfuns.stored_hycom_dates()
fn_miss_list = ocnfuns.get_missing_file_num_list(hy_dates)



print(len(hy_dates))
print(fn_miss_list)
fn2 = np.array(fn_miss_list)
ind0 = np.where(fn2 == 0)[0]
print(ind0)
keeper = np.max(ind0)

cnt = 0
for dt in hy_dates:
    if cnt != keeper:
        print('the date ' + dt + ' will be deleted')
        fnsd = hycom_dir + '*' + dt + 'T12:00_2*.nc'
        fls2d = glob.glob(fnsd)
        print(fls2d)
        print('will be deleted')
    else:
        print('the date ' + dt + ' will not be deleted')
    cnt=cnt+1

